필요 패키지


In [7]:
from urllib import request
from bs4 import BeautifulSoup
import pandas as pd
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
from requests import Session
import re
from tqdm import tqdm
import datetime
import time
import random
from kafka import KafkaProducer
import schedule

## 종목별 코드

In [2]:
ACODE = {"호텔신라":"008770","두산퓨얼셀":"336260","한화솔루션":"009830",
            "에코프로":"086520","신성이엔지":"011930","유니슨":"018000",
            "카카오":"035720","네이버":"035420","엔씨소프트":"036570",
            "sds":"018260","삼성전자":"005930","SK하이닉스":"000660",
            "skc":"011790","한솔케미칼":"014680","DB하이텍":"000990",
            "롯데쇼핑":"023530","신세계":"004170","삼성물산":"028260",
            "동서":"026960","kb금융":"105560","한국금융지주":"071050","신한지주":"055550",
            "우리금융지주":"316140","하나금융지주":"086790",
         "넥슨지티":"041140","한빛소프트":"047080","코세스":"089890",
      "드래곤플라이":"030350","에이트원":"230980","엔텔스":"069410"}

In [3]:
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params=params if method.upper() == 'GET' else '',
                               data=params if method.upper() == 'POST' else '',
                               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60)  # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit - 1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)

    return resp

## naver_crawl 함수 정의

In [6]:

TIME_MAPPING = {}

i = 1
START = datetime.datetime(2022,1,11,9,0) # test
# # START = datetime.datetime.now() # 9시 시작 스케줄링
s = START
while s.strftime('%H:%M') < "16:05":
    e = s + datetime.timedelta(minutes = 4)
    TIME_MAPPING[i] = (s, e)

    s += datetime.timedelta(minutes=5)
    i += 1
    
def naver_crawl(current, ya):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
    pages = [1]
    #ya=dict()

   
    
    #매핑해주는 숫자를 입력 받아서, 9시 10분을 타임스탑에 <- current 매핑수가 3일 때
    
    # 9시부터 
    TIME_STOP = TIME_MAPPING[current]
    
    

    for acode in ACODE.values():
        TF = False
        title_result = []
        
        for page in pages:
            if TF:
                break
            pages.append(page + 1)
            url = 'https://finance.naver.com/item/board.nhn?code={}&page={}'.format(acode,page)
            resp = download(url, headers = headers)
            dom = BeautifulSoup(resp.content, 'lxml')
            table = dom.find('table', {'class' : 'type2'})
            tt = table.select('tbody > tr')

            for k in tt[2:]:
                K = k.select('td > span')
                if len(K) > 0:
                    date = K[0].text
                    #print(date[-5:], TIME_STOP[0].strftime('%H:%M'),TIME_STOP[1].strftime('%H:%M'))
                    if date[-5:] <= TIME_STOP[0].strftime('%H:%M') or date[-5:] >= TIME_STOP[1].strftime('%H:%M'):
            
                        TF = True
                        break

                    title = k.select('td.title > a')[0]['title']
                    view =K[1].text
                    view = int(view)
                    put_data = (date,title,view)

                                         # 'pos':pos,
                                         #  'neg':neg

                    
                    title_result.append(put_data)

        ya[acode] = title_result

    return ya


In [6]:

bootstrap_servers = ["localhost:9092"]  # kafka broker ip
topicName = 'NAVER_test'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)


def msg():
    #스케줄링
    result = dict()
    current = 1
    while True:
        start = time.time()
        result = naver_crawl(current,result)
        end = time.time()
        print(end-start)
        if TIME_MAPPING[current][1].strftime('%H:%M') <= datetime.datetime.now().strftime('%H:%M'):
            producer.send(topicName, str(result).encode())
            current+=1
            result=dict()
        if current == 85:
            break
        #tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        #producer.send(topicName, tim.encode())
schedule.every().day.at("09:20").do(msg)
while True:
    
    schedule.run_pending()
    

25.584041118621826
26.169064044952393
26.423624992370605
25.64248776435852
26.421719789505005
26.390974044799805
26.57172393798828
27.45395517349243
26.503793954849243
26.719302892684937
26.414897918701172
27.511533737182617
26.499542951583862
26.223752975463867
25.70737385749817
25.63137698173523
25.94360089302063
26.600942850112915
30.039296865463257
27.544144868850708
28.27935266494751
25.93415117263794
30.01923418045044
26.087436199188232
26.17323613166809
26.67684292793274
26.281229972839355
25.756274938583374
25.62726902961731
26.933072090148926
26.373979806900024
26.838332891464233
26.76478099822998
25.73805069923401
26.052881002426147
26.730590343475342
25.85049080848694
25.649302005767822
26.07638907432556
26.402658224105835
25.985461950302124
26.81107497215271
27.192500114440918
26.68400001525879
25.65846085548401
26.041393995285034
25.496098041534424
25.6291401386261
25.54528832435608
26.31792116165161
25.461230039596558


KeyboardInterrupt: 

## producer code

In [ ]:

#20분 -> 25분까지 25분 30초에 다시 돌기 시작 -> 
bootstrap_servers = ['localhost:9092']  # kafka broker ip
topicName = 'NAVER'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

job = schedule.every(60).seconds.do(message)
while True:
    
    NOW = datetime.datetime.now()
    schedule.run_pending()
    if datetime.datetime.now() == NOW + datetime.timedelta(minutes = 1):

        producer.send(topicName, str(job()).encode())

        tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        producer.send(topicName, tim.encode())
        break
